# <center> Google News Web Scrapper With Sentiment Analysis </center>

<div style="text-align: justify"> 
- The purpose of this project is to show my capability of developing a full stack data project 
</div>
<div style="text-align: justify"> 
- The project utilizes python [Web Scrapper, Data Cleaning, Sentiment Analysis and Connection Point to SQL Database],   Microsft SQL Server Management Studio [Data Storage], PowerBi [Visualization] and general problem solving and troubleshooting skill.
</div>

- The project is divided into multiple parts which are:
1. Data Scrapping [Python - Selenium & Beautiful Soup]
2. Data Cleaning and formating [Python - Pandas]
3. Sentiment Analysis [Python - TextBlob]
4. Data Storage [MSSMS]
5. Data Visualization [PowerBi]

### <center> Data Scrapping

In [140]:
#Importing Libraries to be use during data scrapping.
#Selenium is used to access google and send the subject matter that is studied.
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options #Future Improvement - headless option

#Beautiful soup is used to scrape the exact information needed from the Google News Page
#The reason why Beautiful soup is used instead of utilizing selenium as it has the same capability was down to ease of use
from bs4 import BeautifulSoup

#Time is used to create a delay between page as additional buffer to selenium WebDriverWait to ensure all information is 
#accessible by beautiful soup.
import time

In [141]:
#Subject of analysis
id = 'Malaysia' # Can be adjusted to be user input instead of predefined variable, currently optimized for country search

#Initialising the web driver and navigating to the google
driver = webdriver.Chrome(ChromeDriverManager().install())
url = 'https://www.google.com/'
driver.get(url)

#Locating the search bar, inputting the subject of analysis, and navigating to the news page
search_bar_ID = 'APjFqb'
WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.ID, search_bar_ID))).send_keys(id)
WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.ID, search_bar_ID))).send_keys(Keys.RETURN)
time.sleep(5)
WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CLASS_NAME, 'O3S9Rb'))).click() #Navigating to news page


In [142]:
current_page = 1 # The current page of news
num_pages = 10  # Specify the number of news page to iterate 

# Identifying variable to store the scrapped information

news_subject_clean = []
news_outlet_clean = []
time_passed_clean = []

# Iterating through the pages using selenium and extracting the information using beautiful soup

while current_page <= num_pages:
    
    page_source = driver.page_source
    soup = BeautifulSoup(page_source, 'html.parser')

    # Find all elements with a specific class
    
    news_subject = soup.find_all(class_="n0jPhd ynAwRc MBeuO nDgy9d")
    time_passed = soup.find_all(class_ = "OSrXXb rbYSKb LfVVr")
    news_outlet = soup.find_all(class_ = "MgUUmf NUnG9d")

    # Due to the google news layout page which has predictive search subcategory of the subject
    # we are required to remove it from our data scrapped
    
    sections = soup.find_all('div', jsname='K9a4Re')
    for section in sections:
        excluded_elements = section.find_all('div', class_='MgUUmf NUnG9d')
        news_outlet = [element for element in news_outlet if element not in excluded_elements]
        
        
    #Storing the data into the pre intialized variables
    
    for i, element in enumerate (news_subject):
        news_subject_clean.append(element.get_text())

    for i, element in enumerate (news_outlet):
        news_outlet_clean.append(element.get_text())

    for i, element in enumerate (time_passed):
        time_passed_clean.append(element.get_text())
        
    #Navigating the next page button
    next_button = driver.find_element(By.ID, "pnnext")
    next_button.click()

    #Iterating the pages
    current_page += 1
    time.sleep(2)

driver.quit() #Closes the browser and driver

### <center> Data Cleaning and formating

In [143]:
#Importing pandas to format the information into a table for further processes
import pandas as pd

#Importing date and time to include the date the information was extracted
from datetime import date

#pd.set_option('display.max_colwidth', None) - to remove character limit when displaying the data frame

#Merging all the information into a data frame
df = pd.DataFrame({'time_passed': time_passed_clean,'news_outlet': news_outlet_clean, 'news_subject': news_subject_clean})

In [144]:
#Removing html pragraph identifier

df['news_subject'] = df['news_subject'].replace(['\n'], '', regex=True)

In [145]:
#Removing the text ago from time passed

df['time_passed'] = df['time_passed'].replace(['ago'], '', regex=True)

#This was done for future improvement of looping the content of the table and replacing with an integer value in hours

In [146]:
#Timestamp of when the information is extracted

df['date'] = date.today()

In [147]:
#Rearranging the columns for ease of processing

df2 = df[['date','time_passed','news_outlet','news_subject']]

### <center> Sentiment Analysis

In [148]:
#Importing textblob for its sentiment analysis capabilities
from textblob import TextBlob

#Other option include HuggingFace, Vader and Flair

In [149]:
#Intialising a variable to store the sentiment analysis
polarity = []

#Looping through the data frame on the 3rd column (news_subject) and appending the outcome into the variable polarity

for i in range (len(df2)):
    polar = ((TextBlob(df2.iloc[i,3])).sentiment.polarity)
    if polar < 0:
        polarity.append('Negative')
    elif polar == 0:
        polarity.append('Neutral')
    else:
        polarity.append('Positive')

In [150]:
#Transforming the polarity variable array into a data frame 

df_polar = pd.DataFrame({'Sentiment':polarity})

In [151]:
#Concatinating the target data frame (df2) and the sentiment polarity dataframe

df_final = pd.concat([df2,df_polar], axis = 1)

### <center> Data Storage

In [152]:
#Importing pyodbc as a connecting point to MSSMS
import pyodbc

#Other option include SQLAlchemy

In [153]:
#In the event that the table dosent exist in the server this query is defined to create the table
create_table_query = """
--Create Table if it does not exist

IF Object_ID('{subject}') IS NULL

CREATE TABLE [pyodbc].[dbo].[{subject}]
(
[date] DATE,
[time_passed] VARCHAR(50),
[news_outlet] VARCHAR(50),
[news_subject] VARCHAR(250),
[sentiment] VARCHAR (30),
)

""".format(subject=id)

In [154]:
#This query is to inser the data into the table
sql_insert = """
INSERT INTO [pyodbc].[dbo].[{subject}](
    [date],
    [time_passed],
    [news_outlet],
    [news_subject],
    [sentiment]
)
VALUES
(
    ?,?,?,?,?
)""".format(subject=id)

In [155]:
#Defining the component of the connection string

DRIVER = '{ODBC Driver 17 for SQL Server}'
SERVER_NAME = 'DESKTOP-50TGEJ8\SQLEXPRESS'
DATABASE_NAME = 'pyodbc'

#This is based on the user server information

In [156]:
#Defining the connection string

CONNECTION_STRING = """
Driver={driver};
Server={server};
Database={database};
Trusted_Connection=yes;
""".format(
    driver=DRIVER,
    server=SERVER_NAME,
    database = DATABASE_NAME
)

In [157]:
#Connect to the database

connection_object = pyodbc.connect(CONNECTION_STRING)

#Create a cursor object (pyodbc specific function)

cursor_object = connection_object.cursor()

In [158]:
#Create Table
cursor_object.execute(create_table_query)

#Commit the table created
cursor_object.commit()

In [159]:
#convert the dataframe into a record set (dictionary) to process for SQL insert
df_records = df_final.values.tolist()

In [160]:
#Inserting Value into table
cursor_object.executemany(sql_insert,df_records)

#Commit the data into the table
cursor_object.commit()

In [161]:
#End the connection to the server
cursor_object.close()
connection_object.close()